# Ecommerce Sales Prediction using Apache Spark
#### Contributor(s): Yash Sethia, Ritesh Kumar, Shubham 
![Snapshot](https://www.edureka.co/blog/wp-content/uploads/2018/09/Picture5-2.png)
----------
### About Apache Spark

*Apache Spark*, written in Scala, is a general-purpose distributed data processing engine. Or in other words: load big data, do computations on it in a distributed way, and then store it. <br/> <br/>
Apache Spark contains libraries for data analysis, machine learning, graph analysis, and streaming live data. Spark is generally faster than *Hadoop*. 
This is because Hadoop writes intermediate results to disk (that is, lots of I/O operations) whereas Spark tries to keep intermediate results in memory 
(that is, in-memory computation) whenever possible. Moreover, Spark offers lazy evaluation of operations and optimizes them just before the final result; 
Sparks maintains a series of transformations that are to be performed without actually performing those operations unless we try to obtain the results. 
This way, Spark is able to find the best path looking at overall transformations required (for example, reducing two separate steps of adding number 5 and
20 to each element of the dataset into just a single step of adding 25 to each element of the dataset, or not actually doing operations on part of the dataset
which will eventually will be filtered out in the final result). 
<br/><br/>
This makes Spark one of the most popular tools for big data analytics currently.

PySpark is an interface for Apache Spark in Python. It not only allows you to write Spark applications using Python APIs, but also provides the PySpark shell for interactively analyzing your data in a distributed environment.

In this Notebook, we have the data of an Ecommerce website with the following fields:
Definitions:
* Email ID of the user
* Address of the user
* Average Session Length of the user
* Time spent by the user on the app
* Time spent by the user on the website
* Length of the membership of the user
* Yearly amount spent by the user

We used Pyspark API of Apache Spark to handle this data to bring out meaningful inferences and try to predict the Amount spent by users using this data

In [ ]:
!pip install pyspark
!pip install findspark

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import *

In [2]:
#dataset = spark.read.csv("./Ecommerce_Customers.csv", inferSchema = True, header = True)
path = ['./Ecommerce_Customers.csv',
        './Ecommerce_Customers1.csv',
        './Ecommerce_Customers2.csv'
       ]
 
    
dataset = spark.read.options(inferSchema = True, header=True).csv(path)
dataset.show(dataset.count())

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|    ryan36@gmail.com|5379 Rhonda Prair...|       32.22729914|13.72862718|    37.99702801|         4.802630631|        613.5993234|
|    chris67@ryan.biz|716 Bush Greens A...|       32.78977262|11.67006592|    37.40874848|         3.414688423|        483.1597208|
|   katie25@gmail.com|6861 Lopez Fork A...|       32.77260993|13.27631301|    36.60077705|         3.462298847|        540.2634004|
|      kyang@diaz.org|223 Love Trail Su...|       34.37425805|15.12699429|    37.15762409|         5.377593584|        765.5184619|
|tadams@contreras....|049 Matthew Terra...|       33.07871721|12.69578975|  

## Data Exploration

In [3]:
dataset.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [4]:
dataset.show(5)

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|    ryan36@gmail.com|5379 Rhonda Prair...|       32.22729914|13.72862718|    37.99702801|         4.802630631|        613.5993234|
|    chris67@ryan.biz|716 Bush Greens A...|       32.78977262|11.67006592|    37.40874848|         3.414688423|        483.1597208|
|   katie25@gmail.com|6861 Lopez Fork A...|       32.77260993|13.27631301|    36.60077705|         3.462298847|        540.2634004|
|      kyang@diaz.org|223 Love Trail Su...|       34.37425805|15.12699429|    37.15762409|         5.377593584|        765.5184619|
|tadams@contreras....|049 Matthew Terra...|       33.07871721|12.69578975|  

In [5]:
dataset.count()

500

In [6]:
dataset.limit(5).toPandas()

,Email,Address,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,ryan36@gmail.com,"5379 Rhonda Prairie Apt. 696Brownland, WA 87027",32.227299,13.728627,37.997028,4.802631,613.599323
1,chris67@ryan.biz,"716 Bush Greens Apt. 098Trevorton, MA 17817-8000",32.789773,11.670066,37.408748,3.414688,483.159721
2,katie25@gmail.com,"6861 Lopez Fork Apt. 114South Jamiechester, DE...",32.772610,13.276313,36.600777,3.462299,540.263400
3,kyang@diaz.org,"223 Love Trail Suite 831Port Jeffrey, IN 46849",34.374258,15.126994,37.157624,5.377594,765.518462
4,tadams@contreras.info,"049 Matthew TerraceLake Matthew, MS 20210",33.078717,12.695790,35.358444,4.001786,553.601535


In [7]:
ds = dataset.select("Email", "Address")
ds.limit(10).toPandas()

,Email,Address
0,ryan36@gmail.com,"5379 Rhonda Prairie Apt. 696Brownland, WA 87027"
1,chris67@ryan.biz,"716 Bush Greens Apt. 098Trevorton, MA 17817-8000"
2,katie25@gmail.com,"6861 Lopez Fork Apt. 114South Jamiechester, DE..."
3,kyang@diaz.org,"223 Love Trail Suite 831Port Jeffrey, IN 46849"
4,tadams@contreras.info,"049 Matthew TerraceLake Matthew, MS 20210"
5,craigcastro@burgess.com,"65407 Warner Forges Suite 071New Lori, AZ 3013..."
6,ellen24@anthony.com,"PSC 3408, Box 3353APO AE 05113-1257"
7,davisrobert@hicks-smith.com,"95967 Pitts BurgsWest Sarah, MH 27817-5147"
8,walkerlaura@peterson-yates.com,"89154 Jones Stream Suite 311East Katie, NC 727..."
9,marc54@hotmail.com,Unit 6174 Box 1949DPO AP 27478


## Descriptive Analytics

In [8]:
# Arraged according to Amount Spent
dataset.orderBy('Yearly Amount Spent', ascending=False).limit(5).toPandas()

,Email,Address,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,kyang@diaz.org,"223 Love Trail Suite 831Port Jeffrey, IN 46849",34.374258,15.126994,37.157624,5.377594,765.518462
1,asilva@yahoo.com,USNV JohnsonFPO AP 19026,34.603311,12.207298,33.913847,6.922689,744.221867
2,william82@gmail.com,"11143 Park SquaresSamanthatown, UT 97073",33.256335,13.858062,37.780265,5.976768,725.584814
3,jeffrey54@mcdonald-williams.com,"297 Francis ValleySouth Lindsey, NY 13669-5367",34.967610,13.919494,37.952013,5.066697,712.396327
4,rhonda01@gmail.com,"939 Watson RunStaceyberg, VT 58376-0454",34.385820,12.729720,36.232110,5.705941,708.935185


In [9]:
# Arraged according to Length of membership
dataset.orderBy('Length of Membership', ascending=False).limit(5).toPandas()

,Email,Address,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,asilva@yahoo.com,USNV JohnsonFPO AP 19026,34.603311,12.207298,33.913847,6.922689,744.221867
1,alicia85@lee.com,"14220 Carla Flat Suite 521Lake Matthew, DE 06183",32.887105,12.387184,37.431159,6.401229,684.163431
2,waltonkaren@gmail.com,"355 Villegas Isle Apt. 070West Jenniferview, N...",35.742670,10.889828,35.565436,6.115199,669.987141
3,ebrown@osborne.com,"4291 Nichols Fork Apt. 562Thomaschester, SC 08...",31.945396,12.965761,36.966389,6.076654,657.019924
4,william82@gmail.com,"11143 Park SquaresSamanthatown, UT 97073",33.256335,13.858062,37.780265,5.976768,725.584814


In [10]:
dataset.describe('Email', 'Address').show()

+-------+-----------------+--------------------+
|summary|            Email|             Address|
+-------+-----------------+--------------------+
|  count|              500|                 500|
|   mean|             null|                null|
| stddev|             null|                null|
|    min|aaron04@yahoo.com|0001 Mack MillNor...|
|    max|zscott@wright.com|Unit 7502 Box 834...|
+-------+-----------------+--------------------+



In [11]:
dataset.agg({'Length of Membership':'mean'}).show()

+-------------------------+
|avg(Length of Membership)|
+-------------------------+
|        3.533461555930001|
+-------------------------+



In [12]:
dataset.agg({'Yearly Amount Spent':'mean'}).show()

+------------------------+
|avg(Yearly Amount Spent)|
+------------------------+
|       499.3140382607999|
+------------------------+



In [13]:
dataset.summary().toPandas()

,summary,Email,Address,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,count,500,500,500,500,500,500,500
1,mean,None,None,33.053193518240015,12.052487936927994,37.06044542108,3.533461555930001,499.3140382607999
2,stddev,None,None,0.9925631111602917,0.9942156084624618,1.0104889068105996,0.9992775024367541,79.31478155115916
3,min,aaron04@yahoo.com,"0001 Mack MillNorth Jennifer, NE 42021-5936",29.53242897,8.508152176,33.91384725,0.26990109,256.6705823
4,25%,None,None,32.33889932,11.38677555,36.33952101,2.926940235,444.9665517
5,50%,None,None,33.07871721,11.98204499,37.06708997,3.53286158,498.6355985
6,75%,None,None,33.71065306,12.75207661,37.71598618,4.125584363,549.1315733
7,max,zscott@wright.com,Unit 7502 Box 8345DPO AE 53747,36.13966249,15.12699429,40.00518164,6.922689335,765.5184619


## Linear Regression
From the above data, we have seen that even after ordering data on the basis of various attributes, there is no direct inference about how Yearly amount spent by user varies with these attributes. So we can use the Linear Regression predictor of Pyspark to predict the Yearly amount spent by using the following attributes as independent features:
* Average Session Length
* Time spent on the App
* Time spent on the Website
* Length of Membership

In [14]:
#Linear Regression on the data to predict the yearly amount spent

from pyspark.ml.regression import LinearRegression

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

featureassembler=VectorAssembler(inputCols=["Avg Session Length","Time on App","Time on Website","Length of Membership"],outputCol="Independent Features")

output=featureassembler.transform(dataset)

output.toPandas()

,Email,Address,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent,Independent Features
0,ryan36@gmail.com,"5379 Rhonda Prairie Apt. 696Brownland, WA 87027",32.227299,13.728627,37.997028,4.802631,613.599323,"[32.22729914, 13.72862718, 37.99702801, 4.8026..."
1,chris67@ryan.biz,"716 Bush Greens Apt. 098Trevorton, MA 17817-8000",32.789773,11.670066,37.408748,3.414688,483.159721,"[32.78977262, 11.67006592, 37.40874848, 3.4146..."
2,katie25@gmail.com,"6861 Lopez Fork Apt. 114South Jamiechester, DE...",32.772610,13.276313,36.600777,3.462299,540.263400,"[32.77260993, 13.27631301, 36.60077705, 3.4622..."
3,kyang@diaz.org,"223 Love Trail Suite 831Port Jeffrey, IN 46849",34.374258,15.126994,37.157624,5.377594,765.518462,"[34.37425805, 15.12699429, 37.15762409, 5.3775..."
4,tadams@contreras.info,"049 Matthew TerraceLake Matthew, MS 20210",33.078717,12.695790,35.358444,4.001786,553.601535,"[33.07871721, 12.69578975, 35.35844431, 4.0017..."
...,...,...,...,...,...,...,...,...
495,lewisjessica@craig-evans.com,"4483 Jones Motorway Suite 872Lake Jamiefurt, U...",33.237660,13.566160,36.417985,3.746573,573.847438,"[33.23765998, 13.56615961, 36.4179848, 3.74657..."
496,katrina56@gmail.com,"172 Owen Divide Suite 497West Richard, CA 19320",34.702529,11.695736,37.190268,3.576526,529.049004,"[34.70252897, 11.69573629, 37.19026771, 3.5765..."
497,dale88@hotmail.com,"0787 Andrews Ranch Apt. 633South Chadburgh, TN...",32.646777,11.499409,38.332576,4.958264,551.620146,"[32.64677668, 11.49940906, 38.33257633, 4.9582..."
498,cwilson@hotmail.com,"680 Jennifer Lodge Apt. 808Brendachester, TX 0...",33.322501,12.391423,36.840086,2.336485,456.469510,"[33.32250105, 12.39142299, 36.84008573, 2.3364..."


In [15]:
output.select("Independent Features").show()

+--------------------+
|Independent Features|
+--------------------+
|[32.22729914,13.7...|
|[32.78977262,11.6...|
|[32.77260993,13.2...|
|[34.37425805,15.1...|
|[33.07871721,12.6...|
|[32.8052204,11.83...|
|[32.43075793,11.3...|
|[32.17909997,11.1...|
|[33.15417579,11.8...|
|[34.33589584,12.2...|
|[32.38625186,10.6...|
|[32.80869759,12.8...|
|[33.87974497,13.5...|
|[32.04983939,12.2...|
|[33.55520742,11.5...|
|[33.14207933,11.4...|
|[32.59718266,10.8...|
|[33.16713688,11.9...|
|[31.51473786,12.5...|
|[34.59402115,10.9...|
+--------------------+
only showing top 20 rows



In [16]:
output.columns

['Email',
 'Address',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent',
 'Independent Features']

In [17]:
finalized_data=output.select("Independent Features","Yearly Amount Spent")
finalized_data.toPandas()

,Independent Features,Yearly Amount Spent
0,"[32.22729914, 13.72862718, 37.99702801, 4.8026...",613.599323
1,"[32.78977262, 11.67006592, 37.40874848, 3.4146...",483.159721
2,"[32.77260993, 13.27631301, 36.60077705, 3.4622...",540.263400
3,"[34.37425805, 15.12699429, 37.15762409, 5.3775...",765.518462
4,"[33.07871721, 12.69578975, 35.35844431, 4.0017...",553.601535
...,...,...
495,"[33.23765998, 13.56615961, 36.4179848, 3.74657...",573.847438
496,"[34.70252897, 11.69573629, 37.19026771, 3.5765...",529.049004
497,"[32.64677668, 11.49940906, 38.33257633, 4.9582...",551.620146
498,"[33.32250105, 12.39142299, 36.84008573, 2.3364...",456.469510


In [18]:
train_data,test_data=finalized_data.randomSplit([0.75,0.25])

regressor=LinearRegression(featuresCol='Independent Features', labelCol='Yearly Amount Spent')
regressor=regressor.fit(train_data)

### Coefficients and Intercept
Linear Regression predicts the Yearly Amount spent (say Y) as a function of independent features (say X1, X2, X3, X4). So linear regression predicts Y as:

Y = W0 + W1 x X1 + W2 x X2 + W3 x X3 + W4 x X4

where, W1, W2, W3, W4 are coefficients or weights for each independent feature and W0 = intercept. <br/>
We can show these values as follows.

In [19]:
regressor.coefficients

DenseVector([25.686, 38.8722, 0.6728, 61.729])

In [20]:
regressor.intercept

-1061.0312921451975

In [21]:
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show(40)

C:\Users\SHUBHAM\anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+-------------------+------------------+
|Independent Features|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[30.83643267,13.1...|        467.5019004| 471.9277912353941|
|[31.06621816,11.7...|        448.9332932|462.10368005281384|
|[31.30919264,11.9...|        432.7207178| 429.8096220443638|
|[31.36621217,11.1...|        430.5888826|427.00831258490007|
|[31.42522688,13.2...|        530.7667187| 535.3804355128404|
|[31.57020083,13.3...|        545.9454921|  564.076929417183|
|[31.65480968,13.0...|        475.2634237|469.19308082046496|
|[31.6739155,12.32...|        475.7250679|502.57959494298143|
|[31.7207699,11.75...|        538.7749335| 546.6590656499122|
|[31.72420252,13.1...|        503.3878873| 509.8444538505289|
|[31.76561882,12.4...|        496.5540816|  501.897724704555|
|[31.81861657,11.2...|        446.4186734| 448.5036677673677|
|[31.82797906,12.4...|        440.0027475|449.70073936793824|
|[31.853

## Performance Analysis

In [22]:
trainingSummary = regressor.summary
print("Root Mean Squared Error on training data: %f" % trainingSummary.rootMeanSquaredError)
print("R Squared (R2) on training data: %f" % trainingSummary.r2)

Root Mean Squared Error on training data: 9.756769
R Squared (R2) on training data: 0.985905


Root Mean Sqaured Error (RMSE) measures the differences between predicted values by the model and the actual values. However, RMSE alone is meaningless until we compare with the actual “Yearly Amount Spent” value, such as mean, min and max. After such comparison, our RMSE looks pretty good.

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                385|
|   mean| 498.72152288857103|
| stddev|  82.28923545535812|
|    min|        256.6705823|
|    max|        765.5184619|
+-------+-------------------+



In [24]:
lr_predictions = regressor.transform(test_data)
lr_predictions.select("prediction","Yearly Amount Spent","Independent features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Yearly Amount Spent",metricName="r2")
r2_test = lr_evaluator.evaluate(lr_predictions)
print("R Squared (R2) on test data = %g" % r2_test)

+------------------+-------------------+--------------------+
|        prediction|Yearly Amount Spent|Independent features|
+------------------+-------------------+--------------------+
| 471.9277912353941|        467.5019004|[30.83643267,13.1...|
|462.10368005281384|        448.9332932|[31.06621816,11.7...|
| 429.8096220443638|        432.7207178|[31.30919264,11.9...|
|427.00831258490007|        430.5888826|[31.36621217,11.1...|
| 535.3804355128404|        530.7667187|[31.42522688,13.2...|
+------------------+-------------------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.976471


R squared at **0.976471** indicates that in our model, approximate **97%** of the variability in “Yearly Amount Spent” can be explained using the regressor.